# Taking Geospatial Data Analytics to the Next Level with Panel, DuckDB and MapLibre

### Introduction
In this blog post, we will explore how to prepare and query data using DuckDB. Next, we will create a custom component from scratch with Python Panel and integrate it with MapLibreGL JS. Finally, we will demonstrate how to dynamically update map data based on the zoom level, step by step.

### Data Analysis
We will use DuckDB to explore the POI (Point of Interest) data shared by Foursquare as open data. We will query and retrieve data for Turkey and generate H3 indices for this data using the DuckDB H3 extension.

### Project Setup

We will create the project structure using Python Uv. We will also install all the required packages to set up the working environment. We install the Uv package manager using the following code:

curl -LsSf https://astral.sh/uv/install.sh | sh

We create the project using the following code:

uv init open-poi
cd open-poi

To set up the virtual environment, we use the following command:

uv venv --python 3.11

We install the required packages for our project using the following commands:

uv add "duckdb>=1.1.3" "panel>=1.5.5"

To include Jupyter Notebook in our development environment, we install it using the following command:

uv add jupyterlab --dev

We create a notebook directory in our project folder and add a notebook file named “poi-data.ipynb” inside it. To install the spatial and H3 extensions for DuckDB:




In [11]:
import duckdb
import os


db_dir = '/Users/ulrike_imac_air/projects/maps/GeospatialDataAnalysisUsingDuckDB/open-poi/data'
db_path = os.path.join(db_dir, 'poi.duckdb')
db = duckdb.connect(db_path, read_only=False)
db.sql("""
INSTALL spatial;
INSTALL h3 FROM community;
LOAD h3;
LOAD spatial;
""")

To load the data shared by Foursquare into DuckDB, use the following SQL queries. These create two separate tables: categories for all categories and places for data filtered to only include locations in Turkey (country = ‘TR’):

In [14]:
db.sql("""
DROP TABLE IF EXISTS categories;
""")

db.sql("""
CREATE TABLE categories AS  
  SELECT * 
  FROM read_parquet('s3://fsq-os-places-us-east-1/release/dt=2025-01-10/categories/parquet/*.zstd.parquet');
""")

In [15]:
db.sql("""
DROP TABLE IF EXISTS places;
""")

db.sql("""
CREATE TABLE places AS  
  SELECT * 
  FROM read_parquet('s3://fsq-os-places-us-east-1/release/dt=2025-01-10/places/parquet/*.zstd.parquet')
  WHERE country = 'LU';
""")